In [16]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import yfinance as yf
import matplotlib.pyplot as pt

# 1. 下载资产数据
client = MongoClient("mongodb://localhost:27017/")
# 选择数据库（例如 "mydatabase"）
db = client["stock_data"]
collection_aapl = db["AAPL"]
collection_googl = db["GOOGL"]
collection_msft = db["MSFT"]

In [17]:
# 2. 设置初始权重
target_weights = {'AAPL': 0.4, 'MSFT': 0.3, 'GOOGL': 0.3}
data = pd.DataFrame({
    'AAPL' : pd.DataFrame(collection_aapl.find())['Close'], 
    'GOOGL' : pd.DataFrame(collection_googl.find())['Close'] ,
    'MSFT' : pd.DataFrame(collection_msft.find())['Close'] 
})

returns = data.pct_change()
returns.index = pd.DataFrame(collection_aapl.find())['Date']

returns.dropna(inplace=True)

In [18]:
initial_cash = 100000
portfolio_value = initial_cash
tickers = ['AAPL','GOOGL','MSFT']
portfolio = {ticker: 0 for ticker in tickers}
values = []
dates = []

In [37]:
returns.index = returns.index.strftime("%Y-%m-%d")

In [21]:
# 5. 回测循环
for date, row in returns.iterrows():
    # 如果是再平衡日
    if date in rebalancing_dates:
        portfolio_value = sum(data.loc[date][ticker] * portfolio[ticker] for ticker in tickers)
        for ticker in tickers:
            allocation = target_weights[ticker] * portfolio_value
            portfolio[ticker] = allocation // data.loc[date][ticker]

    # 更新投资组合价值
    portfolio_value = sum(data.loc[date][ticker] * portfolio[ticker] for ticker in tickers)
    values.append(portfolio_value)
    dates.append(date)

# 6. 绘图
pt.figure(figsize=(12, 6))
pt.plot(dates, values, label='Portfolio Value')
pt.title('Rebalancing Portfolio Strategy')
pt.xlabel('Date')
pt.ylabel('Portfolio Value ($)')
pt.legend()
pt.grid()
pt.show()

KeyError: Timestamp('2022-01-04 05:00:00')